In [32]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [33]:
def generate_sample_sales_data(start_date, num_days=14, num_products=18, num_locations=3):
    dates = [(start_date + timedelta(days=x)).strftime('%Y-%m-%d') 
             for x in range(num_days)]
    data = []
    for date in dates:
        for product_id in range(1, num_products + 1):
            row = {
                'date': date,
                'product_id': f'PROD_{product_id}',
                'daily_sales': np.random.randint(50, 200),
                'inventory_level': np.random.randint(200, 1000),
                'unit_price': round(np.random.uniform(10, 100), 2),
                'lead_time_days': np.random.randint(2, 7),
                'is_promotion': np.random.choice([0, 1], p=[0.8, 0.2])
            }
            data.append(row)
    return data

In [34]:
start_date = datetime(2021, 1, 1)
sample_df = generate_sample_sales_data(start_date)

In [35]:
print(sample_df)

[{'date': '2021-01-01', 'product_id': 'PROD_1', 'daily_sales': 151, 'inventory_level': 800, 'unit_price': 10.48, 'lead_time_days': 4, 'is_promotion': np.int64(0)}, {'date': '2021-01-01', 'product_id': 'PROD_2', 'daily_sales': 183, 'inventory_level': 537, 'unit_price': 22.57, 'lead_time_days': 2, 'is_promotion': np.int64(1)}, {'date': '2021-01-01', 'product_id': 'PROD_3', 'daily_sales': 96, 'inventory_level': 952, 'unit_price': 60.03, 'lead_time_days': 5, 'is_promotion': np.int64(0)}, {'date': '2021-01-01', 'product_id': 'PROD_4', 'daily_sales': 109, 'inventory_level': 694, 'unit_price': 34.19, 'lead_time_days': 4, 'is_promotion': np.int64(0)}, {'date': '2021-01-01', 'product_id': 'PROD_5', 'daily_sales': 171, 'inventory_level': 871, 'unit_price': 99.97, 'lead_time_days': 4, 'is_promotion': np.int64(0)}, {'date': '2021-01-01', 'product_id': 'PROD_6', 'daily_sales': 198, 'inventory_level': 502, 'unit_price': 61.31, 'lead_time_days': 6, 'is_promotion': np.int64(0)}, {'date': '2021-01-01',

In [ ]:
'date': date,
'product_id': f'PROD_{product_id}',
'daily_sales': np.random.randint(50, 200),
'inventory_level': np.random.randint(200, 1000),
'unit_price': round(np.random.uniform(10, 100), 2),
'lead_time_days': np.random.randint(2, 7),
'is_promotion': np.random.choice([0, 1], p=[0.8, 0.2])